# UEP 239 In-Class Activity 
### Vector data in Python 

---
Kyle Monahan, Chris Barnett, Patrick Florance | April 2021

The goal of this activity is to provide an introduction to how to load spatial data in  Python. In this session, we will cover:


* Reading and writing spatial data; how to join tabular and vector data
* Checking coordinate systems with geopandas
* Calculating basic operations on vector data (area, distance)
* Performing spatial joins for vector data
* Visualizing vector data with geopandas, matplotlib, and shapely
* Geocoding addresses

---

## Getting started with the data 

No matter what type of data we are working with, we need to start by importing libraries and loading the data.

## Importing libraries 

We can import libraries with the code below. We do this once per session. 

In [ ]:
import numpy as np               # Load numpy, for scientific computing in Python - matrices are your friend here, like MATLAB https://numpy.org/
                                 # You can even run Fortran and C++ code in numpy
    
import pandas as pd              # Load pandas, for data frames - note these are just the same as R data frames - similar to dplyr/plyr 
import matplotlib.pyplot as plt  # Load matplotlib for plotting and 
import matplotlib.image as mpimg # For plotting image (raster) data

%matplotlib inline               
# ^ Remind Jupyter to give us the results of matplotlib inline (right below the cell) - this is called Jupyter magic  

#!pip install seaborn
#!pip install seaborn --upgrade

import seaborn as sns            # Seaborn is similar to matplotlib, but has additional features and a slightly different approach 

#### Loading and working with a shapefile

Just as before, we will load and work with a shapefile. Note that the ☆ means this section will be covered in the homework, and you should review this code when working on the homework. 

### ☆ Reading and writing spatial data; how to join tabular and vector data


For this work, we will be loading a few datasets:

1. AirBnB data for New York City in 2019 in tabular form (CSV)
2. Bussiness data from Reference USA in tabular form (CSV)
3. Bouroughs vector data from GitHub in geojson format

To do this, we will have to learn how to join tabular and vector data. 

### Loading the data 

To load the CSV data, we can use **pandas**, just as we did previously.

#### Dataset 1 - New York AirBnB data

For our first dataset, we are working with the New York AirBnB data. This was originally drawn from Kaggle in 2019: https://www.kaggle.com/dgomonov/new-york-city-airbnb-open-data

If you look into the source of this data, it's actually just InsideAirBnB, but I used this version since they have done some cleaning for us already, and we already discussed pandas data cleaning. Plus you can review lots of approaches to doing this in Python on Kaggle. 

PROTIP: Always check out the source of any new datasets and especially any metadata. 

In [ ]:
import os 
os.getcwd()
# If you need to change your working directory, you can do that with the below
#os.chdir(r'C:\Users\kmonah02\Box\Classes\Uku_Class_Guest_Lecture\UEP239\VectorData\3_InClass')

In [ ]:
airbnbNYC = pd.read_csv('Data/AB_NYC_2019.csv')
airbnbNYC.head(2)

In [ ]:
airbnbNYC.info()

In [ ]:
airbnbNYC.value_counts()

In [ ]:
airbnbNYC.describe()

#### Dataset 2 - Reference USA data

Next, we will check out the ReferenceUSA datasets. See the link here for more information:

https://sites.tufts.edu/datalab/tufts-subscriptionsholdings/#ReferenceUSA

#### Steps to download this data on your own
Login to Tufts VPN or Virtual Lab > Navigate to the Tufts catalog link at the link above > Click US Bussinesses Database > Advanced Search > Select Verified and Unverfied > Select New York, NY > Keyword SIC NAICS > Search for "Health" and Find Health Food Grocers 44511007 > Select all records > Download > Download CSV.



> **BREAKOUT GROUP ACTIVITY (20 minutes)**: Give the above steps a try in groups to download a sample dataset. Try different keyword searches to see what appears! We will use an example dataset for class today, but this may be useful for your projects.


#### Loading the data

In [ ]:
import geopandas as gpd
foods_NYC = gpd.read_file('Data/NYC_HealthFoods.csv')
foods_NYC.head(2)

Note how we can read this column in as a geopandas dataframe, but it is missing the **geometry** column. Why would this be?


#### Dataset 3 Bouroughs data 


In [ ]:
bour = gpd.read_file("https://raw.githubusercontent.com/codeforamerica/click_that_hood/master/public/data/new-york-city-boroughs.geojson")
bour.head(2)

# Please don't run this over and over since we are pulling from github 

Unlike the previous dataset, the geojson file has a geometry column. Let's look at the raw data to see why. 

https://raw.githubusercontent.com/codeforamerica/click_that_hood/master/public/data/new-york-city-boroughs.geojson

> **BREAKOUT GROUP ACTIVITY (10 minutes)**: Click on the link above. What is different about this data that allows for geopandas to "know" the geometry of the shapes? Look into the geopandas documentation on loading GEOJSON files, and state where in the GEOJSON file you can see that information. How is it stored?


### ☆ Checking coordinate systems with geopandas

Before we think about spatial joins, we need to check to see what coordinate system (CRS) the files are in.

In [ ]:
bour.crs  # WGS 1984 

In [ ]:
airbnbNYC.crs

Note how the airbnb data is missing a **crs** attribute - it is only of class dataframe, not geodataframe. We can confirm this as follows:

In [ ]:
type(airbnbNYC)

In [ ]:
type(bour)

### ☆ Performing attribute joins for vector data 

So it's clear we need to add **geographic information** to the vector data in order to work with it. To do that, we can *join* the data. There are two main ways to join data in geopandas: 

1) attribute joins 
2) spatial joins. 

See the documentation here: https://geopandas.org/docs/user_guide/mergingdata.html

PROTIP: Always look at the *documentation* while you are coding! A few minutes reading the documentation before you start can save hours of headaches. I normally only do this after the hours of headaches, but you don't need to.

> **BREAKOUT GROUPS (10 min):** What is the difference between a spatial join and an attribute join? When do you use each one? 


In [ ]:
airbnbNYC.merge(bour,left_on='neighbourhood', right_on='name')

# Note - you'll use a similar approach to join tabular data on your homework, and it does work there!

But this failed! Why? Let's look at the unique values in each.

In [ ]:
airbnbNYC.neighbourhood.unique()

In [ ]:
bour.name.unique()

In [ ]:
# We can also use .isin() to check if 
# these values are in the other column 

airbnbNYC.isin(bour.name).describe()

If we knew anything about New York City's geography, we would know that the bouroughs and the neighbourhoods do not align! So what can we do? Luckily, with geopandas, we can add back the geographic data using lat and long values! 
#### Convert from xy (lat long) inside CSV to geopandas data frame

In [ ]:
airbnbNYC = gpd.GeoDataFrame(
    airbnbNYC, geometry=gpd.points_from_xy(airbnbNYC.longitude, airbnbNYC.latitude))
airbnbNYC.head(2)

**QUESTION:** How do we know that this worked, looking at the table above?

In [ ]:
airbnbNYC.crs # Returns... nothing? That's because it is not defined! 
airbnbNYC = airbnbNYC.set_crs('epsg:4326')

airbnbNYC.plot(column='price')


**QUESTION:** What is wrong with the above graph? What can we do to fix it?

##### An example approach of how to figure out what is wrong

In [ ]:
airbnbNYC.price.max()

In [ ]:
airbnbNYC.price.min()

In [ ]:
airbnbNYC.price.mean()

In [ ]:
airbnbNYC = airbnbNYC[airbnbNYC['price'] < 9999]
airbnbNYC['price'] = airbnbNYC['price'].replace({0:np.nan})

In [ ]:
airbnbNYC.price.max()

In [ ]:
airbnbNYC['log_price'] = np.log(airbnbNYC['price'])
airbnbNYC.plot(column='log_price')

### ☆ Performing spatial joins for vector data
Now we need to spatially join these data. 

> **BREAKOUT GROUP ACTIVITY (10 minutes)**: Find an example of a spatial join using the gpd.sjoin() command online, and look at the commands. Tip - look at the geopandas documentation.

**QUESTIONS:** Any questions on what we covered so far?

In [ ]:
airbnbNYC.crs == bour.crs

In [ ]:
# Only shared rooms
shared_only = airbnbNYC[airbnbNYC['room_type'] == 'Shared room']
joined_data = gpd.sjoin(bour, shared_only, how='inner', op='contains', lsuffix='left', rsuffix='right')
joined_data.head(2)

In [ ]:
joined_data.shape

### ☆ Calculating basic operations on vector data (area, distance)

Next, we may wish to find the area of each of these polygon observations.

Note, what do I need to do to the joined_data in order to correctly calculate area?

In [ ]:
joined_data = joined_data.to_crs('epsg:2263') # Long Island State Plain
joined_data['area'] = joined_data.area

We can also calculate a new column:

In [ ]:
joined_data['log_price_per_area'] = joined_data.log_price / joined_data.area
joined_data.log_price_per_area.max()

### ☆ Geocoding addresses

Finally, we may need to look up place names or addresses in a geocoder.

A geocoder will convert a place name e.g. Tufts University to a latitude and longitude location. 

> **BREAKOUT GROUP ACTIVITY (35 minutes)**: Learn more about the geocoders we will use to complete this class and for the next homework. 

Nominatim geocoder policy: 
https://operations.osmfoundation.org/policies/nominatim/

Geocode.farm policy:
https://geocode.farm/geocoding/free-api-documentation/

Think about the following questions:
1. What are the limits for our use?
2. If Tufts has a single IP address, why does it matter if we have a 250 requests per day per IP address? 
3. Why do these services require a user string, commonly populated with an email? 

Produce a summary of your thoughts to share with the group.

#### How to geocode our health food data 

We don't have geographic information on the foods_NYC data, but we do have addresses. We need to geocode those addresses.

In [ ]:
foods_NYC.head(2)


In [ ]:
foods_NYC["Full Address"] = foods_NYC['Address'] + foods_NYC["City"] + foods_NYC['State'] + foods_NYC['ZIP Code']

In [ ]:
foods_NYC.shape

In [ ]:
foods_NYC.columns.tolist()[110:130]

In [ ]:
foods_NYC['Home Business'].unique()

In [ ]:
foods_NYC = foods_NYC[foods_NYC['Home Business'] == 'No']

In [ ]:
foods_NYC.shape

In [ ]:
foods_NYC['Year Established'].unique()

In [ ]:
#foods_NYC = foods_NYC[foods_NYC['Year Established'] == '2018']
#foods_NYC.shape

In [ ]:
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut

found_locations = []
lat = []
long = []

# Change this string!
geolocator = Nominatim(user_agent='kyle.monahan@tufts.edu')
for address in foods_NYC["Full Address"].iteritems():
    try:
        location = geolocator.geocode(address[1])
        print("The location is",location)
        found_locations.append(location)
        if location is not None:
            lat.append(location[1][0])
            long.append(location[1][1])
        else:
            lat.append(None)
            long.append(None)
    except GeocoderTimedOut as e:
        print("Error: geocode failed on input %s with message %s"%(my_address, e.message))


### We made it!
We made it! Now we are ready to get started with spatial data in Python.

To summarize what we learned: 

* Reading and writing spatial data; how to join tabular and vector data
* Checking coordinate systems with geopandas
* Calculating basic operations on vector data (area, distance)
* Performing spatial joins for vector data
* Visualizing vector data with geopandas, matplotlib, and shapely
* Geocoding addresses

### Time to work on HW5

For the rest of the class, you may work through HW5, posted on Canvas and Piazza at the link below:
https://piazza.com/tufts/spring2021/uep239/resources